In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
#from PIL import Image
import os
from keras import layers


#Data is downloaded into the container via shellscript. We believe this is more efficient
traindirectory="./archive/FaceMaskDataset/Train"
testdirectory="./archive/FaceMaskDataset/Test"
validationdirectory="./archive/FaceMaskDataset/Validation"
image_size=224
TrainData=keras.utils.image_dataset_from_directory(traindirectory, class_names=["WithoutMask","WithMask"], image_size=(image_size,image_size))
TestData=keras.utils.image_dataset_from_directory(testdirectory, class_names=["WithoutMask","WithMask"], image_size=(image_size,image_size))
ValidationData=keras.utils.image_dataset_from_directory(validationdirectory, class_names=["WithoutMask","WithMask"], image_size=(image_size,image_size))

def normalization(image, label):
    return tf.cast(image, tf.float32)/255., label

TrainData=TrainData.map(normalization)
TestData=TestData.map(normalization)

from datetime import datetime
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

early_stop= tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=2
)


#with strategy.scope():
#    model=build_model(2, 224)
'''
model.fit(TrainData,
        epochs=1,
          validation_data=TestData
         , callbacks=[tboard_callback,early_stop])
'''

Found 10000 files belonging to 2 classes.
Found 992 files belonging to 2 classes.
Found 800 files belonging to 2 classes.


'\nmodel.fit(TrainData,\n        epochs=1,\n          validation_data=TestData\n         , callbacks=[tboard_callback,early_stop])\n'

In [2]:
from tensorflow.keras import datasets, layers, models

model2 = models.Sequential()
model2.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.Flatten())
model2.add(layers.Dense(64, activation='relu'))
model2.add(layers.Dense(2, activation="softmax", dtype='float32'))
model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
model2.fit(TrainData,
        epochs=1,
          validation_data=TestData)

313/313 [==============================] - 40s 112ms/step - loss: 0.1645 - accuracy: 0.9458 - val_loss: 0.3680 - val_accuracy: 0.8851


In [3]:
import tensorflow_model_optimization as tfmot

quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model2)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

q_aware_model.fit(TrainData, epochs=2, validation_data=TestData)

Epoch 1/2
313/313 [==============================] - 58s 178ms/step - loss: 0.0687 - accuracy: 0.9758 - val_loss: 0.0623 - val_accuracy: 0.9819
Epoch 2/2
313/313 [==============================] - 56s 178ms/step - loss: 0.0724 - accuracy: 0.9765 - val_loss: 0.1411 - val_accuracy: 0.9577
